### Carregando e criando um tratamento para retirada das linhas aonde o ID é nulo

In [130]:
import pandas as pd

df = pd.read_csv("../data/raw_data/compra_produtos.csv", sep=";", encoding="ansi")

df.dropna(subset=["compra_id", "pedido_id"], inplace=True)

### Tratando as colunas de datas

In [131]:
df["data_pedido"] = pd.to_datetime(df["data_pedido"], errors='coerce')
df["data_recebimento"] = pd.to_datetime(df["data_recebimento"], errors='coerce')
df["data_producao"] = pd.to_datetime(df["data_producao"], errors='coerce')
df["data_validade"] = pd.to_datetime(df["data_validade"], errors='coerce')

### Ajustando as colunas de valores para float e tirando poluição dos dados

In [132]:
df["preco_unit_compra"] = df["preco_unit_compra"].str.replace(",",".").str.replace(r"\.(?=.*[^\.])","")
df["preco_unit_compra"] = df["preco_unit_compra"].astype(float)

df["frete"] = df["frete"].str.replace(",",".").str.replace(r"\.(?=.*\.)","")
df["frete"] = df["frete"].astype(float).fillna(0.0)

df["desconto"] = df["desconto"].str.replace(",",".").str.replace(r"\.(?=.*\.)","")
df["desconto"] = df["desconto"].astype(float).fillna(0.0)


df["custo_total"] = df["custo_total"].str.replace(",",".").str.replace(r"\.(?=.*\.)","")
df["custo_total"] = df["custo_total"].astype(float)


### Como os dados são de compras já feitas e não pode comprar sem nota estou removendo as linhas que estão sem notas e transformando as que sobraram em inteiro

In [133]:

df.dropna(subset=["nfe"], inplace=True)
df["nfe"] = df["nfe"].astype(int)

### Paronizando colunas do tipo texto

In [134]:
df["fornecedor_nome"] = df["fornecedor_nome"].str.lower().str.strip().str.replace(" ","_")
df["descricao_fornecedor"] = df["descricao_fornecedor"].str.lower().str.strip().str.replace(" ","_")
df["marca"] = df["marca"].str.lower().str.strip().str.replace(" ","_")
df["area"] = df["area"].str.lower().str.strip().str.replace(" ","_")
df["loja"] = df["loja"].str.lower().str.strip().str.replace(" ","_")

### Verificando se existem datas de recebimento menores do que as datas do pedido e datas de validade menores que a data de produção, caso existam vou removelas pois não faz sentido e provavelmente foi pedido errado

In [135]:
#Verifica se a data de recebimento é menor do que a data de envio se for exclui a linha
df["validacao_datas"] = (df["data_pedido"].isna()) | (df["data_recebimento"].isna()) | (df["data_recebimento"] >= df["data_pedido"])

datas_erradas = df[df["validacao_datas"] == False].index
df = df.drop(datas_erradas)
df = df.drop("validacao_datas", axis=1)


#Agora a mesma logica para as datas de validade e produção
df["validacao_datas"] = (df["data_producao"].isna()) | (df["data_validade"].isna()) | (df["data_validade"] >= df["data_producao"])

datas_erradas = df[df["validacao_datas"] == False].index
df = df.drop(datas_erradas)
df = df.drop("validacao_datas", axis=1)

In [136]:
df.to_parquet("../data/processed_data/compra_produtos.parquet", index=False)